# Prediction of Formation Energy of Solids Using Neural Network


This work was reported by Jha *et al.*[1] and in this exercise, we will implement, train, and validate a model to predict the formation energy of given solid using its formula as the input.


1. Jha, D., Ward, L., Paul, A., Liao, W.-K., Choudhary, A., Wolverton, C., & Agrawal, A. **(2018)**. ElemNet : Deep Learning the Chemistry of Materials From Only Elemental Composition. Scientific Reports, 8(1), 17593. http://doi.org/10.1038/s41598-018-35934-y